In [3]:
nsls2=False

In [46]:
import binstar_client
from binstar_client import Binstar
from argparse import Namespace
import os
import subprocess
from pprint import pprint
if nsls2:
    # Fill in your binstar token here
    token = 'ed-a09a18b1-8bce-4582-b02b-d43c27db57bd'
    username = 'nsls2-tag'
    site = 'https://pergamon.cs.nsls2.local:8443/api'
    os.environ['REQUESTS_CA_BUNDLE']='/etc/certificates/ca_cs_nsls2_local.crt'
else:
    token=None
    username = 'lightsource2-dev'
    site = None

    

cli = binstar_client.utils.get_binstar(Namespace(token=token, site=site))

Using Anaconda Cloud api site https://api.anaconda.org


In [68]:
def get_file_names_on_anaconda_channel(username, channel='main'):
    return set([f['basename'] for f in cli.show_channel('main', username)['files']])

In [133]:
def determine_build_name(path_to_recipe, *conda_build_args):
    if conda_build_args is None:
        conda_build_args = []
    cmd = ['conda', 'build', path_to_recipe, '--output']
    cmd.extend(conda_build_args)
    ret = subprocess.check_output(cmd, stderr=subprocess.STDOUT).decode().strip().split('\n')
    if len(ret) > 1:
        # Then this is the first time we are getting the build name and conda
        # has to check out the source. Call it a second time and you get the full
        # path to the file that will be spit out by conda-build
        return determine_build_name(path_to_recipe, *conda_build_args)
    cmd.remove('--output')
    return ret[0], cmd

In [134]:
recipes_folder = '/home/edill/dev/conda/staged-recipes-dev/recipes'
build_names = []
print("Determining package build names...")
for folder in sorted(os.listdir(recipes_folder)):
    for pyver in ['2.7', '3.4', '3.5']:
        name, build_cmd = determine_build_name(os.path.join(recipes_folder, folder), '--python', pyver)
        print(name)
        build_names.append((os.sep.join(name.split(os.sep)[-2:]), build_cmd))

Determining package build names...
/home/edill/mc/conda-bld/linux-64/album-0.0.2.post0-0_g6b05c00_py27.tar.bz2
/home/edill/mc/conda-bld/linux-64/album-0.0.2.post0-0_g6b05c00_py34.tar.bz2
/home/edill/mc/conda-bld/linux-64/album-0.0.2.post0-0_g6b05c00_py35.tar.bz2
/home/edill/mc/conda-bld/linux-64/bluesky-0.4.0rc2.post23-23_gd9d1f42_py27.tar.bz2
/home/edill/mc/conda-bld/linux-64/bluesky-0.4.0rc2.post23-23_gd9d1f42_py34.tar.bz2
/home/edill/mc/conda-bld/linux-64/bluesky-0.4.0rc2.post23-23_gd9d1f42_py35.tar.bz2
/home/edill/mc/conda-bld/linux-64/channelarchiver-0.0.5.6.post0-0_gfac126f_py27.tar.bz2
/home/edill/mc/conda-bld/linux-64/channelarchiver-0.0.5.6.post0-0_gfac126f_py34.tar.bz2
/home/edill/mc/conda-bld/linux-64/channelarchiver-0.0.5.6.post0-0_gfac126f_py35.tar.bz2
/home/edill/mc/conda-bld/linux-64/conda-build-1.19.0.post30-30_g4e4808d_py27.tar.bz2
/home/edill/mc/conda-bld/linux-64/conda-build-1.19.0.post30-30_g4e4808d_py34.tar.bz2
/home/edill/mc/conda-bld/linux-64/conda-build-1.19.0.p

In [137]:
lightsource2_packages

{'linux-64/boltons-16.0.0.post12-12_ge853701_py27.tar.bz2',
 'linux-64/boltons-16.0.0.post12-12_ge853701_py34.tar.bz2',
 'linux-64/boltons-16.0.0.post12-12_ge853701_py35.tar.bz2',
 'linux-64/boltons-16.0.0.post13-13_g11f3c15_py27.tar.bz2',
 'linux-64/boltons-16.0.0.post13-13_g11f3c15_py34.tar.bz2',
 'linux-64/boltons-16.0.0.post13-13_g11f3c15_py35.tar.bz2',
 'linux-64/boltons-16.0.0.post4-4_gc394b86.tar.bz2',
 'linux-64/boltons-16.0.0.post6-6_g968841a.tar.bz2',
 'linux-64/boltons-16.0.0.post8-8_g6d186bf.tar.bz2',
 'linux-64/boltons-16.0.0.post8-8_g6d186bf_py27.tar.bz2',
 'linux-64/boltons-16.0.0.post8-8_g6d186bf_py34.tar.bz2',
 'linux-64/boltons-16.0.0.post8-8_g6d186bf_py35.tar.bz2',
 'linux-64/channelarchiver-0.0.5.post6-6_gfac126f_np19py27.tar.bz2',
 'linux-64/channelarchiver-0.0.5.post6-6_gfac126f_np19py34.tar.bz2',
 'linux-64/channelarchiver-0.0.5.post6-6_gfac126f_np19py35.tar.bz2',
 'linux-64/channelarchiver-0.0.5.post6-6_gfac126f_npNonepy35.tar.bz2',
 'linux-64/channelarchiver-0.

In [138]:
lightsource2_packages = get_file_names_on_anaconda_channel('lightsource2-dev')
build_package = []
dont_build_package = []
for name, cmd in build_names:
    if name in lightsource2_packages:
        dont_build_package.append((name, cmd))
    else:
        build_package.append((name, cmd))

print("%s / %s packages already exist on %s" % (
        len(dont_build_package), 
        len(dont_build_package) + len(build_package),
        username))
if dont_build_package:
    for pkg, cmd in dont_build_package:
        print(pkg)
else:
    print("No packages exist on %s" % username)
print()

print("%s / %s packages do not exist on %s" % (
        len(build_package), 
        len(dont_build_package) + len(build_package),
        username))
if build_package:
    for pkg, cmd in build_package:
        print(pkg)
else:
    print("No packages to be built.")
    
# print("%s: %s" % (build_this_package, build_name))


6 / 54 packages already exist on lightsource2-dev
linux-64/datamuxer-0.3.0.post0-0_g35b9b71_py27.tar.bz2
linux-64/datamuxer-0.3.0.post0-0_g35b9b71_py34.tar.bz2
linux-64/datamuxer-0.3.0.post0-0_g35b9b71_py35.tar.bz2
linux-64/doct-1.0.1.post7-7_gb811d2a_py27.tar.bz2
linux-64/doct-1.0.1.post7-7_gb811d2a_py34.tar.bz2
linux-64/doct-1.0.1.post7-7_gb811d2a_py35.tar.bz2

48 / 54 packages do not exist on lightsource2-dev
linux-64/album-0.0.2.post0-0_g6b05c00_py27.tar.bz2
linux-64/album-0.0.2.post0-0_g6b05c00_py34.tar.bz2
linux-64/album-0.0.2.post0-0_g6b05c00_py35.tar.bz2
linux-64/bluesky-0.4.0rc2.post23-23_gd9d1f42_py27.tar.bz2
linux-64/bluesky-0.4.0rc2.post23-23_gd9d1f42_py34.tar.bz2
linux-64/bluesky-0.4.0rc2.post23-23_gd9d1f42_py35.tar.bz2
linux-64/channelarchiver-0.0.5.6.post0-0_gfac126f_py27.tar.bz2
linux-64/channelarchiver-0.0.5.6.post0-0_gfac126f_py34.tar.bz2
linux-64/channelarchiver-0.0.5.6.post0-0_gfac126f_py35.tar.bz2
linux-64/conda-build-1.19.0.post30-30_g4e4808d_py27.tar.bz2
linux-64

In [ ]:
for pkg, cmd in build_package[:2]:
    print("Building %s" % pkg)
    print("Subprocess command: %s" % pkg)
    ret = subprocess.check_output(cmd, stderr=subprocess.STDOUT).decode().strip()
    print(ret)

In [66]:
lightsource2

{'linux-64/boltons-16.0.0.post12-12_ge853701_py27.tar.bz2',
 'linux-64/boltons-16.0.0.post12-12_ge853701_py34.tar.bz2',
 'linux-64/boltons-16.0.0.post12-12_ge853701_py35.tar.bz2',
 'linux-64/boltons-16.0.0.post13-13_g11f3c15_py27.tar.bz2',
 'linux-64/boltons-16.0.0.post13-13_g11f3c15_py34.tar.bz2',
 'linux-64/boltons-16.0.0.post13-13_g11f3c15_py35.tar.bz2',
 'linux-64/boltons-16.0.0.post4-4_gc394b86.tar.bz2',
 'linux-64/boltons-16.0.0.post6-6_g968841a.tar.bz2',
 'linux-64/boltons-16.0.0.post8-8_g6d186bf.tar.bz2',
 'linux-64/boltons-16.0.0.post8-8_g6d186bf_py27.tar.bz2',
 'linux-64/boltons-16.0.0.post8-8_g6d186bf_py34.tar.bz2',
 'linux-64/boltons-16.0.0.post8-8_g6d186bf_py35.tar.bz2',
 'linux-64/channelarchiver-0.0.5.post6-6_gfac126f_np19py27.tar.bz2',
 'linux-64/channelarchiver-0.0.5.post6-6_gfac126f_np19py34.tar.bz2',
 'linux-64/channelarchiver-0.0.5.post6-6_gfac126f_np19py35.tar.bz2',
 'linux-64/channelarchiver-0.0.5.post6-6_gfac126f_npNonepy35.tar.bz2',
 'linux-64/channelarchiver-0.

In [52]:
name = os.path.splitext(os.path.splitext(build_names[1])[0])[0]
cli.search(name)

[]